In [1]:
# imports
import pandas as pd
import StringIO
import zipfile
import requests
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import re
import json

In [2]:
# retrieve CitiBike data June 2015
r = requests.get('https://s3.amazonaws.com/tripdata/201506-citibike-tripdata.zip')
z = zipfile.ZipFile(StringIO.StringIO(r.content))
df = pd.read_csv(z.open('201506-citibike-tripdata.csv'))
df.head()


,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,1338,6/1/2015 0:00,6/1/2015 0:22,128,MacDougal St & Prince St,40.727103,-74.002971,2021,W 45 St & 8 Ave,40.759291,-73.988597,20721,Subscriber,1984,1
1,290,6/1/2015 0:00,6/1/2015 0:05,438,St Marks Pl & 1 Ave,40.727791,-73.985649,312,Allen St & E Houston St,40.722055,-73.989111,21606,Subscriber,1997,1
2,634,6/1/2015 0:01,6/1/2015 0:11,383,Greenwich Ave & Charles St,40.735238,-74.000271,388,W 26 St & 10 Ave,40.749718,-74.002950,16595,Subscriber,1993,1
3,159,6/1/2015 0:01,6/1/2015 0:04,361,Allen St & Hester St,40.716059,-73.991908,531,Forsyth St & Broome St,40.718939,-73.992663,16949,Subscriber,1981,1
4,1233,6/1/2015 0:02,6/1/2015 0:22,382,University Pl & E 14 St,40.734927,-73.992005,532,S 5 Pl & S 4 St,40.710451,-73.960876,17028,Customer,NaN,0


In [3]:
# inspect data
print df.isnull().sum()
print '\n'
print df.shape
print '\n'
print df.columns.values
print '\n'
print df.dtypes

tripduration                    0
starttime                       0
stoptime                        0
start station id                0
start station name              0
start station latitude          0
start station longitude         0
end station id                  0
end station name                0
end station latitude            0
end station longitude           0
bikeid                          0
usertype                        0
birth year                 130392
gender                          0
dtype: int64


(941219, 15)


['tripduration' 'starttime' 'stoptime' 'start station id'
 'start station name' 'start station latitude' 'start station longitude'
 'end station id' 'end station name' 'end station latitude'
 'end station longitude' 'bikeid' 'usertype' 'birth year' 'gender']


tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude  

In [4]:
# read in station data
r = requests.get('https://www.citibikenyc.com/stations/json')
data = json.loads(r.text)

In [5]:
# look at data
data['stationBeanList'][0]

{u'altitude': u'',
 u'availableBikes': 3,
 u'availableDocks': 36,
 u'city': u'',
 u'id': 72,
 u'landMark': u'',
 u'lastCommunicationTime': u'2015-12-28 03:03:43 PM',
 u'latitude': 40.76727216,
 u'location': u'',
 u'longitude': -73.99392888,
 u'postalCode': u'',
 u'stAddress1': u'W 52 St & 11 Ave',
 u'stAddress2': u'',
 u'stationName': u'W 52 St & 11 Ave',
 u'statusKey': 1,
 u'statusValue': u'In Service',
 u'testStation': False,
 u'totalDocks': 39}

In [6]:
# create a loop to build out a dataframe
stations = []
station_stats = {'altitude': None,
 'availableBikes': None,
 'availableDocks': None,
 'city': None,
 'id': None,
 'landMark': None,
 'lastCommunicationTime': None,
 'latitude': None,
 'location': None,
 'longitude': None,
 'postalCode': None,
 'stAddress1': None,
 'stAddress2': None,
 'stationName': None,
 'statusKey': None,
 'statusValue': None,
 'testStation': None,
 'totalDocks': None}
for i in data['stationBeanList']:
    station_stats['altitude'] = i['altitude']
    station_stats['availableBikes'] = i['availableBikes']
    station_stats['availableDocks'] = i['availableDocks']
    station_stats['city'] = i['city']
    station_stats['id'] = i['id']
    station_stats['landMark'] = i['landMark']
    station_stats['lastCommunicationTime'] = i['lastCommunicationTime']
    station_stats['latitude'] = i['latitude']
    station_stats['location'] = i['location']
    station_stats['longitude'] = i['longitude']
    station_stats['postalCode'] = i['postalCode']
    station_stats['stAddress1'] = i['stAddress1']
    station_stats['stAddress2'] = i['stAddress2']
    station_stats['stationName'] = i['stationName']
    station_stats['statusKey'] = i['statusKey']
    station_stats['statusValue'] = i['statusValue']
    station_stats['testStation'] = i['testStation']
    station_stats['totalDocks'] = i['totalDocks']
    stations.append(station_stats.copy())

cols = ['altitude','availableBikes','availableDocks','city','id','landMark','lastCommunicationTime','latitude','longitude','postalCode','stAddress1','stAddress2','stationName','statusKey','testStation','totalDocks']
df_stations = pd.DataFrame(stations,columns=cols)

In [7]:
# check shape
df_stations.shape

(506, 16)

In [8]:
# rename stationName column to match
df_stations.columns = ['altitude', 'availableBikes', 'availableDocks', 'city', 'id',
       'landMark', 'lastCommunicationTime', 'latitude', 'longitude',
       'postalCode', 'stAddress1', 'stAddress2', 'start station name',
       'statusKey', 'testStation', 'totalDocks']

In [9]:
# merge the two dataframes together
df.columns.values
df_stations.columns.values
df_total = df.merge(df_stations,how='left',on='start station name')
df_total.isnull().sum()

tripduration                    0
starttime                       0
stoptime                        0
start station id                0
start station name              0
start station latitude          0
start station longitude         0
end station id                  0
end station name                0
end station latitude            0
end station longitude           0
bikeid                          0
usertype                        0
birth year                 130392
gender                          0
altitude                    26147
availableBikes              26147
availableDocks              26147
city                        26147
id                          26147
landMark                    26147
lastCommunicationTime       26147
latitude                    26147
longitude                   26147
postalCode                  26147
stAddress1                  26147
stAddress2                  26147
statusKey                   26147
testStation                 26147
totalDocks    

In [10]:
# drop nulls
df_total.dropna(subset=['totalDocks'],inplace=True)
df_total.isnull().sum()

tripduration                    0
starttime                       0
stoptime                        0
start station id                0
start station name              0
start station latitude          0
start station longitude         0
end station id                  0
end station name                0
end station latitude            0
end station longitude           0
bikeid                          0
usertype                        0
birth year                 126688
gender                          0
altitude                        0
availableBikes                  0
availableDocks                  0
city                            0
id                              0
landMark                        0
lastCommunicationTime           0
latitude                        0
longitude                       0
postalCode                      0
stAddress1                      0
stAddress2                      0
statusKey                       0
testStation                     0
totalDocks    

In [11]:
df_total.shape

(915072, 30)

In [13]:
# create csv file
df_total.to_csv('C:/Users/Michael/projects/capitalbikeshare/June2015CitiBike.csv')